In [1]:
import datetime
import nltk
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import time
import math
import sys
import gensim
import os
import warnings
import torch
import itertools
import argparse
import shlex
import random
import multiprocessing as mp
from collections import Counter, defaultdict
from inspect import signature
from scipy.stats import ks_2samp, hypergeom, pearsonr, spearmanr
from sklearn.metrics import precision_recall_curve, f1_score, auc
from sklearn.model_selection import train_test_split, KFold
from scipy import spatial, stats
from statsmodels.sandbox.stats.multicomp import multipletests
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
from gensim.utils import simple_preprocess
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.cluster import AgglomerativeClustering
from nltk.corpus import brown, stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

sys.path.append("../../oats")
from oats.utils.utils import save_to_pickle, load_from_pickle, flatten, to_hms
from oats.utils.utils import function_wrapper_with_duration, remove_duplicates_retain_order
from oats.biology.dataset import Dataset
from oats.biology.groupings import Groupings
from oats.biology.relationships import ProteinInteractions, AnyInteractions
from oats.annotation.ontology import Ontology
from oats.annotation.annotation import annotate_using_noble_coder
from oats.distances import pairwise as pw
from oats.nlp.vocabulary import get_overrepresented_tokens, get_vocab_from_tokens
from oats.nlp.vocabulary import reduce_vocab_connected_components, reduce_vocab_linares_pontes

from _utils import Method
from _utils import IndexedGraph


# Some settings for how data is visualized in the notebook.
mpl.rcParams["figure.dpi"] = 400
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Warming up PyWSD (takes ~10 secs)... took 6.746164798736572 secs.


In [2]:
# Create and name an output directory according to when the notebooks or script was run.
name = "topic_modeling"
OUTPUT_DIR = os.path.join("../outputs","{}_{}_{}".format(name,datetime.datetime.now().strftime('%m_%d_%Y_h%Hm%Ms%S'),random.randrange(1000,9999)))
os.mkdir(OUTPUT_DIR)

In [3]:
# Paths to different datasets containing gene names, text descriptions, and/or ontology term annotations.
plant_dataset_path = "../../plant-data/genes_texts_annots.csv"
clinvar_dataset_path = "../data/clinvar/clinvar_diseases.csv"
snpedia_snippets_dataset_path = "../data/snpedia/snpedia_snippets.csv"
snpedia_contexts_dataset_path = "../data/snpedia/snpedia_contexts.csv"

# Paths to datasets of sentence or description pairs.
paired_phenotypes_path = "../data/paired_sentences/plants/scored.csv"
biosses_datset_path = "../data/paired_sentences/biosses/cleaned_by_me.csv"

# Paths to files for data about how genes can be grouped into biochemical pathways, etc.
kegg_pathways_path = "../../plant-data/reshaped_data/kegg_pathways.csv" 
plantcyc_pathways_path = "../../plant-data/reshaped_data/plantcyc_pathways.csv" 
lloyd_meinke_subsets_path = "../../plant-data/reshaped_data/lloyd_meinke_subsets.csv" 
lloyd_meinke_classes_path = "../../plant-data/reshaped_data/lloyd_meinke_classes.csv" 

# Paths files that contain mappings from the identifiers used by those groups to full name strings.
kegg_pathways_names_path = "../../plant-data/reshaped_data/kegg_pathways_name_map.csv"
plantcyc_pathways_names_path = "../../plant-data/reshaped_data/plantcyc_pathways_name_map.csv"
lloyd_meinke_subsets_names_path = "../../plant-data/reshaped_data/lloyd_meinke_subsets_name_map.csv"
lloyd_meinke_classes_names_path = "../../plant-data/reshaped_data/lloyd_meinke_classes_name_map.csv"

# Paths to other files including the ortholog edgelist from Panther, and cleaned files from the two papers.
pppn_edgelist_path = "../../plant-data/papers/oellrich_walls_et_al_2015/supplemental_files/13007_2015_53_MOESM9_ESM.txt"
ortholog_file_path = "../../plant-data/databases/panther/PlantGenomeOrthologs_IRB_Modified.txt"
lloyd_function_hierarchy_path = "../../plant-data/papers/lloyd_meinke_2012/versions_cleaned_by_me/192393Table_S1_Final.csv"

In [4]:
# Pathways to text corpora files that are used in this analysis.
background_corpus_filename = "../data/corpus_related_files/untagged_text_corpora/background.txt"
phenotypes_corpus_filename = "../data/corpus_related_files/untagged_text_corpora/phenotypes_all.txt"

In [5]:
# Paths to pretrained or saved models used for embeddings with Word2Vec or Doc2vec.
doc2vec_plants_path = "../models/plants_dbow/doc2vec.model"
doc2vec_wikipedia_path = "../models/enwiki_dbow/doc2vec.bin"
word2vec_plants_path = "../models/plants_sg/word2vec.model"
word2vec_wikipedia_path = "../models/wiki_sg/word2vec.bin"

# Paths to BioBERT models.
biobert_pmc_path = "../models/biobert_v1.0_pmc/pytorch_model"                                  
biobert_pubmed_path = "../models/biobert_v1.0_pubmed/pytorch_model"                                 
biobert_pubmed_pmc_path = "../models/biobert_v1.0_pubmed_pmc/pytorch_model"      

# Word2Vec models availalbe pretrained from Pyysalo et al.
# http://bio.nlplab.org/#doc-tools
# http://evexdb.org/pmresources/vec-space-models/
word2vec_bio_pmc_path = "../models/bio_nlp_lab/PMC-w2v.bin"
word2vec_bio_pubmed_path = "../models/bio_nlp_lab/PubMed-w2v.bin"
word2vec_bio_pubmed_and_pmc_path = "../models/bio_nlp_lab/PubMed-and-PMC-w2v.bin"
word2vec_bio_wikipedia_pubmed_and_pmc_path = "../models/bio_nlp_lab/wikipedia-pubmed-and-PMC-w2v.bin"

<a id="part_7"></a>
# Part 7. Topic Modeling
The purpose of this section is to look at different ways that the embeddings obtained for the dataset of phenotype descriptions can be used to cluster or organize the genes to which those phenotypes are mapped into subgroups or representations. These approaches include generating topic models from the data, and doing agglomerative clustering to find clusters to which each gene belongs.

In [7]:
# Rereading in the datasets used by this section so that it can be run independently of other notebook sections.
dataset = Dataset(plant_dataset_path)
dataset.filter_has_description()
lloyd_meinke_subsets_name_mapping = {row.group_id:row.group_name for row in pd.read_csv(lloyd_meinke_subsets_names_path).itertuples()}
groups = Groupings(lloyd_meinke_subsets_path, lloyd_meinke_subsets_name_mapping)
id_to_group_ids, group_id_to_ids = groups.get_groupings_for_dataset(dataset)

<a id="compare_to_subsets"></a>
### Comparing topics learned by topic modeling to existing categorizations
Topic modelling learns a set of word probability distributions from the dataset of text descriptions, which represent distinct topics which are present in the dataset. Each text description can then be represented as a discrete probability distribution over the learned topics based on the probability that a given piece of text belongs to each particular topics. This is a form of data reduction because a high dimensionsal bag-of-words can be represented as a vector of *k* probabilities where *k* is the number of topics. The main advantages of topic modelling over clustering is that topic modelling provides soft classifications that can be additionally interpreted, rather than hard classifications into a single cluster. Topic models are also explainable, because the word probability distributions for that topic can be used to determine which words are most representative of any given topic. One problem with topic modelling is that is uses the n-grams embeddings to semantic similarity between different words is not accounted for. To help alleviate this, this section uses implementations of some existing algorithms to compress the vocabulary as a preprocessing step based on word distance matrices generated using word embeddings.

Topic models define topics present in a dataset of texts as word or n-gram probability distributions. These models represent each instance of text then as being composed of or generated as as mixture of these topics. The vector for each text that indicates which fraction of that text is generated by a each topic is of length *n* where *n* is the number of topics, and can be used as a reduced dimensionality of the text, with a much smaller vector length than the n-grams embedding itself. Therefore we can build a topic model of the data with 100 topics for example in order to then represent each description in the dataset as a a vector of length 100. This section constructs topic models from the n-gram representations of the dataset and selects different values for the number of topics in order to find a value that works well during the grid search over the training dataset.

In [8]:
# Gene IDs are used in this section, so we want to map gene IDs to fully preprocessed descriptions.
descriptions = dataset.get_description_dictionary()
preprocessed_descriptions = {i:" ".join(preprocess_string(d)) for i,d in descriptions.items()}
texts = list(preprocessed_descriptions.values())

# Basic parameters for this problem that are currently used.
number_of_topics = 42
seed = 0

In [9]:
# Creating and fitting the topic model, either NFM or LDA or something like that.
vectorizer = TfidfVectorizer(max_features=10000, stop_words="english", max_df=0.95, min_df=2, lowercase=True)
features = vectorizer.fit_transform(texts)
cls = NMF(n_components=number_of_topics, random_state=seed)
cls.fit(features)

# Function for retrieving the topic vectors for a list of text descriptions.
def get_topic_embeddings(texts, model, vectorizer):
    ngrams_vectors = vectorizer.transform(texts).toarray()
    topic_vectors = model.transform(ngrams_vectors)
    return(topic_vectors)

# Create the dataframe containing the average score assigned to each topic for the genes from each subset.
group_to_topic_vector = {}
for group_id,ids in group_id_to_ids.items():
    texts = [preprocessed_descriptions[i] for i in ids]
    topic_vectors = get_topic_embeddings(texts, cls, vectorizer)
    mean_topic_vector = np.mean(topic_vectors, axis=0)
    group_to_topic_vector[group_id] = mean_topic_vector
    
# Turning that matrix of weights into a dataframe so it can be worked with.
tm_df = pd.DataFrame(group_to_topic_vector)

# Changing the order of the Lloyd, Meinke phenotype subsets to match other figures and tables for consistency.
lmtm_df = pd.read_csv(lloyd_function_hierarchy_path)    
columns_in_order = [col for col in lmtm_df["Subset Symbol"].values if col in tm_df.columns]
columns_in_order.reverse()
assert len(columns_in_order) == number_of_topics
tm_df = tm_df[columns_in_order]
    
# Reordering so consistency with the curated subsets can be checked by looking at the diagonal.
tm_df["idxmax"] = tm_df.idxmax(axis = 1)
tm_df["idxmax"] = tm_df["idxmax"].apply(lambda x: tm_df.columns.get_loc(x))
tm_df = tm_df.sort_values(by="idxmax")
tm_df.drop(columns=["idxmax"], inplace=True)

# Saving a version of this dataframe this is indexed by topic integers and subset strings, before makings topics a column instead.
topic_subset_similarity_df = tm_df
tm_df = tm_df.reset_index(drop=False).rename({"index":"topic"},axis=1).reset_index(drop=False).rename({"index":"order"},axis=1)

In [10]:
topic_subset_similarity_df.head(10)

,OBI,PTH,MCH,CHS,HRM,NUT,MPH,MEC,LIT,TMP,WAT,CPR,PRA,CUL,TCM,RTH,STT,MTM,CDR,SEN,FLT,SSC,SRF,OVP,FSM,MSL,ARC,IST,LEF,ROT,GRS,PIG,NLS,GER,SRL,NHM,MSD,EMB,MGD,EMG,GEM,GAM
24,0.028782,0.038346,0.018190,0.019017,0.011971,0.005932,0.003360,0.010461,0.004570,0.013092,0.014622,0.004480,0.003958,0.002071,0.002515,0.003159,0.002124,0.001743,0.002252,0.003467,0.004644,0.004293,0.002933,0.002248,0.002543,0.010321,0.003353,0.003930,0.005820,0.004901,0.006861,0.002779,0.005213,0.003172,0.003220,0.000271,0.002072,0.001816,0.003463,0.002627,0.004465,0.002163
19,0.017163,0.058757,0.006772,0.005603,0.004673,0.002318,0.002980,0.010081,0.004655,0.007192,0.004079,0.000677,0.006748,0.003958,0.003973,0.003271,0.003142,0.004165,0.000121,0.005048,0.001513,0.000276,0.003625,0.003509,0.003004,0.033775,0.002492,0.001560,0.004524,0.003181,0.007035,0.003115,0.004096,0.001343,0.003633,0.011215,0.000992,0.001369,0.001880,0.001352,0.003402,0.000699
13,0.004004,0.005362,0.007893,0.015528,0.047807,0.010528,0.017414,0.002014,0.003800,0.008723,0.023467,0.003128,0.003656,0.001751,0.003010,0.003408,0.010155,0.002685,0.001804,0.004582,0.003773,0.007906,0.002570,0.002352,0.001736,0.006016,0.002722,0.002635,0.003080,0.004212,0.003703,0.002770,0.004243,0.025743,0.003078,0.002545,0.002676,0.002899,0.001825,0.001158,0.000681,0.001743
32,0.003024,0.002981,0.004288,0.004323,0.010303,0.004980,0.008996,0.008122,0.043353,0.005526,0.005276,0.002107,0.003522,0.006321,0.021468,0.021208,0.008708,0.019964,0.031187,0.013205,0.020995,0.018499,0.016760,0.002305,0.017927,0.019143,0.019052,0.032378,0.021616,0.032221,0.017867,0.011229,0.035786,0.013834,0.006927,0.003410,0.006187,0.006434,0.013361,0.007307,0.003944,0.006953
18,0.010267,0.018306,0.067269,0.086090,0.058027,0.047617,0.053231,0.002361,0.023762,0.065814,0.087628,0.012285,0.007476,0.006774,0.008401,0.005054,0.007599,0.001733,0.008774,0.015282,0.012308,0.017340,0.007584,0.003313,0.003473,0.013087,0.004039,0.008000,0.007571,0.015410,0.012008,0.008009,0.009930,0.024181,0.004449,0.007887,0.003567,0.004611,0.016066,0.006881,0.002681,0.001998
1,0.000136,0.003058,0.006288,0.002149,0.003307,0.003507,0.000172,0.003457,0.003410,0.003496,0.003845,0.009261,0.005421,0.003600,0.003618,0.002014,0.003042,0.003350,0.001846,0.001869,0.002446,0.003984,0.002016,0.002416,0.002033,0.002602,0.001991,0.001901,0.001966,0.002360,0.002334,0.002698,0.001914,0.002250,0.003117,0.000264,0.004143,0.001111,0.003262,0.002258,0.003902,0.001197
15,0.001417,0.001389,0.001405,0.002626,0.001860,0.003435,0.000990,0.001334,0.023546,0.008963,0.003140,0.024537,0.002726,0.002310,0.001305,0.004089,0.001765,0.000889,0.018938,0.003608,0.005998,0.004538,0.002134,0.000720,0.001932,0.003637,0.003307,0.003860,0.003429,0.003694,0.004878,0.007021,0.006085,0.004627,0.005508,0.002234,0.000803,0.000859,0.001517,0.001741,0.000512,0.000330
39,0.011620,0.016277,0.016476,0.016313,0.012922,0.032430,0.012431,0.007162,0.015588,0.039235,0.026104,0.028494,0.094719,0.015735,0.010941,0.012506,0.010669,0.012490,0.011503,0.020994,0.015042,0.055904,0.011502,0.019099,0.007434,0.022151,0.010993,0.014318,0.013995,0.012682,0.018826,0.019963,0.009614,0.038119,0.016600,0.007786,0.030590,0.013844,0.011119,0.009077,0.006430,0.008171
41,0.005562,0.002092,0.011937,0.002681,0.008378,0.004462,0.009672,0.006139,0.005958,0.006320,0.003258,0.009076,0.005314,0.019452,0.018898,0.011660,0.016946,0.013138,0.008362,0.005428,0.008892,0.016325,0.009910,0.011089,0.018950,0.006110,0.016315,0.010186,0.015235,0.011694,0.009791,0.005767,0.013599,0.005381,0.004777,0.010029,0.006155,0.005830,0.011047,0.008981,0.002276,0.002090
29,0.011093,0.007820,0.009037,0.006578,0.008696,0.005053,0.005386,0.022774,0.009056,0.005598,0.007765,0.006005,0.007515,0.042000,0.038515,0.034399,0.029837,0.006975,0.004020,0.013968,0.009436,0.014066,0.019651,0.038290,0.026271,0.015911,0.021113,0.024804,0.018879,0.020304,0.014963,0.011634,0.022172,0.008700,0.014642,0.006824,0.027841,0.019658,0.013679,0.015000,0.007195,0.011254


In [11]:
tm_df.head(10)

,order,topic,OBI,PTH,MCH,CHS,HRM,NUT,MPH,MEC,LIT,TMP,WAT,CPR,PRA,CUL,TCM,RTH,STT,MTM,CDR,SEN,FLT,SSC,SRF,OVP,FSM,MSL,ARC,IST,LEF,ROT,GRS,PIG,NLS,GER,SRL,NHM,MSD,EMB,MGD,EMG,GEM,GAM
0,0,24,0.028782,0.038346,0.018190,0.019017,0.011971,0.005932,0.003360,0.010461,0.004570,0.013092,0.014622,0.004480,0.003958,0.002071,0.002515,0.003159,0.002124,0.001743,0.002252,0.003467,0.004644,0.004293,0.002933,0.002248,0.002543,0.010321,0.003353,0.003930,0.005820,0.004901,0.006861,0.002779,0.005213,0.003172,0.003220,0.000271,0.002072,0.001816,0.003463,0.002627,0.004465,0.002163
1,1,19,0.017163,0.058757,0.006772,0.005603,0.004673,0.002318,0.002980,0.010081,0.004655,0.007192,0.004079,0.000677,0.006748,0.003958,0.003973,0.003271,0.003142,0.004165,0.000121,0.005048,0.001513,0.000276,0.003625,0.003509,0.003004,0.033775,0.002492,0.001560,0.004524,0.003181,0.007035,0.003115,0.004096,0.001343,0.003633,0.011215,0.000992,0.001369,0.001880,0.001352,0.003402,0.000699
2,2,13,0.004004,0.005362,0.007893,0.015528,0.047807,0.010528,0.017414,0.002014,0.003800,0.008723,0.023467,0.003128,0.003656,0.001751,0.003010,0.003408,0.010155,0.002685,0.001804,0.004582,0.003773,0.007906,0.002570,0.002352,0.001736,0.006016,0.002722,0.002635,0.003080,0.004212,0.003703,0.002770,0.004243,0.025743,0.003078,0.002545,0.002676,0.002899,0.001825,0.001158,0.000681,0.001743
3,3,32,0.003024,0.002981,0.004288,0.004323,0.010303,0.004980,0.008996,0.008122,0.043353,0.005526,0.005276,0.002107,0.003522,0.006321,0.021468,0.021208,0.008708,0.019964,0.031187,0.013205,0.020995,0.018499,0.016760,0.002305,0.017927,0.019143,0.019052,0.032378,0.021616,0.032221,0.017867,0.011229,0.035786,0.013834,0.006927,0.003410,0.006187,0.006434,0.013361,0.007307,0.003944,0.006953
4,4,18,0.010267,0.018306,0.067269,0.086090,0.058027,0.047617,0.053231,0.002361,0.023762,0.065814,0.087628,0.012285,0.007476,0.006774,0.008401,0.005054,0.007599,0.001733,0.008774,0.015282,0.012308,0.017340,0.007584,0.003313,0.003473,0.013087,0.004039,0.008000,0.007571,0.015410,0.012008,0.008009,0.009930,0.024181,0.004449,0.007887,0.003567,0.004611,0.016066,0.006881,0.002681,0.001998
5,5,1,0.000136,0.003058,0.006288,0.002149,0.003307,0.003507,0.000172,0.003457,0.003410,0.003496,0.003845,0.009261,0.005421,0.003600,0.003618,0.002014,0.003042,0.003350,0.001846,0.001869,0.002446,0.003984,0.002016,0.002416,0.002033,0.002602,0.001991,0.001901,0.001966,0.002360,0.002334,0.002698,0.001914,0.002250,0.003117,0.000264,0.004143,0.001111,0.003262,0.002258,0.003902,0.001197
6,6,15,0.001417,0.001389,0.001405,0.002626,0.001860,0.003435,0.000990,0.001334,0.023546,0.008963,0.003140,0.024537,0.002726,0.002310,0.001305,0.004089,0.001765,0.000889,0.018938,0.003608,0.005998,0.004538,0.002134,0.000720,0.001932,0.003637,0.003307,0.003860,0.003429,0.003694,0.004878,0.007021,0.006085,0.004627,0.005508,0.002234,0.000803,0.000859,0.001517,0.001741,0.000512,0.000330
7,7,39,0.011620,0.016277,0.016476,0.016313,0.012922,0.032430,0.012431,0.007162,0.015588,0.039235,0.026104,0.028494,0.094719,0.015735,0.010941,0.012506,0.010669,0.012490,0.011503,0.020994,0.015042,0.055904,0.011502,0.019099,0.007434,0.022151,0.010993,0.014318,0.013995,0.012682,0.018826,0.019963,0.009614,0.038119,0.016600,0.007786,0.030590,0.013844,0.011119,0.009077,0.006430,0.008171
8,8,41,0.005562,0.002092,0.011937,0.002681,0.008378,0.004462,0.009672,0.006139,0.005958,0.006320,0.003258,0.009076,0.005314,0.019452,0.018898,0.011660,0.016946,0.013138,0.008362,0.005428,0.008892,0.016325,0.009910,0.011089,0.018950,0.006110,0.016315,0.010186,0.015235,0.011694,0.009791,0.005767,0.013599,0.005381,0.004777,0.010029,0.006155,0.005830,0.011047,0.008981,0.002276,0.002090
9,9,29,0.011093,0.007820,0.009037,0.006578,0.008696,0.005053,0.005386,0.022774,0.009056,0.005598,0.007765,0.006005,0.007515,0.042000,0.038515,0.034399,0.029837,0.006975,0.004020,0.013968,0.009436,0.014066,0.019651,0.038290,0.026271,0.015911,0.021113,0.024804,0.018879,0.020304,0.014963,0.011634,0.022172,0.008700,0.014642,0.006824,0.027841,0.019658,0.0136

In [12]:
# Describing what the most representative tokens for each topic in the model are.
num_top_words = 5
map_top_words = {}
feature_names = vectorizer.get_feature_names()
for i,topic_vec in enumerate(cls.components_):
    top_words = []
    print(i,end=": ")
    for fid in topic_vec.argsort()[-1:-num_top_words-1:-1]:
        word = feature_names[fid]
        # The next line is applicable if words in the topic model are actually a function of the words in the texts.
        #word = " ".join(unreduce[word])
        top_words.append(word)
        print(word, end=" ")  
    map_top_words[i] = top_words
    print()

0: embryo termin stage phenotyp defect 
1: visibl phenotyp observ homozygot describ 
2: seedl white palegreen luteu viresc 
3: type wild compar condit hypocotyl 
4: albino pigment class embryo endosperm 
5: endosperm kernel opaqu collaps small 
6: globular embryo defect stage arrest 
7: preglobular defect stage late occur 
8: male steril anther exert glum 
9: seed white development occur mutant 
10: pale green matur pigment seedl 
11: transit embryo defect stage heart 
12: leav rosett dwarf small curl 
13: aba germin stomat insensit hypersensit 
14: root hair later elong primari 
15: fluoresc red wave chlorophyl long 
16: leaf glossi surfac stripe appear 
17: variabl termin embryo phenotyp defect 
18: sensit stress salt drought osmot 
19: lesion suscept infect fungal chlorot 
20: reduc fertil siliqu size seed 
21: flower earli late dai delai 
22: kernel describ wholli atyp improperli 
23: gametophyt femal male defect complet 
24: resist kanamycin similar morpholog bacteri 
25: growth s

In [14]:
# Create a column that specifies what the top tokens for each topic are.
tm_df["tokens"] = tm_df["topic"].map(lambda x: "|".join(map_top_words[x]))

# Move that column to the left for readability before writing to the file.
tokens_col = tm_df.pop("tokens")
tm_df.insert(2, "tokens", tokens_col)

# Renaming the topics to be in order, to be more helpful when preparing figures that are more intuitive.
tm_df["topic_renumbered"] = tm_df["order"].values[::-1]+1
topic_renumbered_col = tm_df.pop("topic_renumbered")
tm_df.insert(2, "topic_renumbered", topic_renumbered_col)

# Remembering a mapping between the topics, their order, and what the renumbered names are.
topic_order_map = {t:i for t,i in zip(tm_df["topic"].values, tm_df["order"].values)}
topic_renumbered_map = {t:i for t,i in zip(tm_df["topic"].values, tm_df["topic_renumbered"].values)}

# Saving this version of the subset and topic similarity data to a file.
tm_df.to_csv(os.path.join(OUTPUT_DIR, "topic_subset_matrix.csv"), index=False)
tm_df.head(10)

,order,topic,topic_renumbered,tokens,OBI,PTH,MCH,CHS,HRM,NUT,MPH,MEC,LIT,TMP,WAT,CPR,PRA,CUL,TCM,RTH,STT,MTM,CDR,SEN,FLT,SSC,SRF,OVP,FSM,MSL,ARC,IST,LEF,ROT,GRS,PIG,NLS,GER,SRL,NHM,MSD,EMB,MGD,EMG,GEM,GAM
0,0,24,42,resist|kanamycin|similar|morpholog|bacteri,0.028782,0.038346,0.018190,0.019017,0.011971,0.005932,0.003360,0.010461,0.004570,0.013092,0.014622,0.004480,0.003958,0.002071,0.002515,0.003159,0.002124,0.001743,0.002252,0.003467,0.004644,0.004293,0.002933,0.002248,0.002543,0.010321,0.003353,0.003930,0.005820,0.004901,0.006861,0.002779,0.005213,0.003172,0.003220,0.000271,0.002072,0.001816,0.003463,0.002627,0.004465,0.002163
1,1,19,41,lesion|suscept|infect|fungal|chlorot,0.017163,0.058757,0.006772,0.005603,0.004673,0.002318,0.002980,0.010081,0.004655,0.007192,0.004079,0.000677,0.006748,0.003958,0.003973,0.003271,0.003142,0.004165,0.000121,0.005048,0.001513,0.000276,0.003625,0.003509,0.003004,0.033775,0.002492,0.001560,0.004524,0.003181,0.007035,0.003115,0.004096,0.001343,0.003633,0.011215,0.000992,0.001369,0.001880,0.001352,0.003402,0.000699
2,2,13,40,aba|germin|stomat|insensit|hypersensit,0.004004,0.005362,0.007893,0.015528,0.047807,0.010528,0.017414,0.002014,0.003800,0.008723,0.023467,0.003128,0.003656,0.001751,0.003010,0.003408,0.010155,0.002685,0.001804,0.004582,0.003773,0.007906,0.002570,0.002352,0.001736,0.006016,0.002722,0.002635,0.003080,0.004212,0.003703,0.002770,0.004243,0.025743,0.003078,0.002545,0.002676,0.002899,0.001825,0.001158,0.000681,0.001743
3,3,32,39,short|hypocotyl|dai|light|height,0.003024,0.002981,0.004288,0.004323,0.010303,0.004980,0.008996,0.008122,0.043353,0.005526,0.005276,0.002107,0.003522,0.006321,0.021468,0.021208,0.008708,0.019964,0.031187,0.013205,0.020995,0.018499,0.016760,0.002305,0.017927,0.019143,0.019052,0.032378,0.021616,0.032221,0.017867,0.011229,0.035786,0.013834,0.006927,0.003410,0.006187,0.006434,0.013361,0.007307,0.003944,0.006953
4,4,18,38,sensit|stress|salt|drought|osmot,0.010267,0.018306,0.067269,0.086090,0.058027,0.047617,0.053231,0.002361,0.023762,0.065814,0.087628,0.012285,0.007476,0.006774,0.008401,0.005054,0.007599,0.001733,0.008774,0.015282,0.012308,0.017340,0.007584,0.003313,0.003473,0.013087,0.004039,0.008000,0.007571,0.015410,0.012008,0.008009,0.009930,0.024181,0.004449,0.007887,0.003567,0.004611,0.016066,0.006881,0.002681,0.001998
5,5,1,37,visibl|phenotyp|observ|homozygot|describ,0.000136,0.003058,0.006288,0.002149,0.003307,0.003507,0.000172,0.003457,0.003410,0.003496,0.003845,0.009261,0.005421,0.003600,0.003618,0.002014,0.003042,0.003350,0.001846,0.001869,0.002446,0.003984,0.002016,0.002416,0.002033,0.002602,0.001991,0.001901,0.001966,0.002360,0.002334,0.002698,0.001914,0.002250,0.003117,0.000264,0.004143,0.001111,0.003262,0.002258,0.003902,0.001197
6,6,15,36,fluoresc|red|wave|chlorophyl|long,0.001417,0.001389,0.001405,0.002626,0.001860,0.003435,0.000990,0.001334,0.023546,0.008963,0.003140,0.024537,0.002726,0.002310,0.001305,0.004089,0.001765,0.000889,0.018938,0.003608,0.005998,0.004538,0.002134,0.000720,0.001932,0.003637,0.003307,0.003860,0.003429,0.003694,0.004878,0.007021,0.006085,0.004627,0.005508,0.002234,0.000803,0.000859,0.001517,0.001741,0.000512,0.000330
7,7,39,35,level|seed|low|acid|content,0.011620,0.016277,0.016476,0.016313,0.012922,0.032430,0.012431,0.007162,0.015588,0.039235,0.026104,0.028494,0.094719,0.015735,0.010941,0.012506,0.010669,0.012490,0.011503,0.020994,0.015042,0.055904,0.011502,0.019099,0.007434,0.022151,0.010993,0.014318,0.013995,0.012682,0.018826,0.019963,0.009614,0.038119,0.016600,0.007786,0.030590,0.013844,0.011119,0.009077,0.006430,0.008171
8,8,41,34,obviou|morpholog|phenotyp|abnorm|observ,0.005562,0.002092,0.011937,0.002681,0.008378,0.004462,0.009672,0.006139,0.005958,0.006320,0.003258,0.009076,0.005314,0.019452,0.018898,0.011660,0.016946,0.013138,0.008362,0.005428,0.008892,0.016325,0.009910,0.011089,0.018950,0.006110,0.016315,0.010186,0.015235,0.011694,0.009791,0.005767,0.013599,0.005381,0.004777,0.010029,0.006155,0.005830,0.011047,0.0

In [16]:
# Producing a version of the previous table that is useful for producing line drawings representing these results.
tm_lines_dict = defaultdict(list)


# Remembering the order of the Lloyd, Meinke phenotype subsets to match other figures for consistency.
lmtm_df = pd.read_csv(lloyd_function_hierarchy_path)   
subset_to_class_map = {s:c for s,c in zip(lmtm_df["Subset Symbol"].values, lmtm_df["Class Name"].values)}
subset_to_desc_map = {s:c for s,c in zip(lmtm_df["Subset Symbol"].values, lmtm_df["Subset Name and Description "].values)}
subset_abbrevs_in_order = [col for col in lmtm_df["Subset Symbol"].values if col in tm_df.columns]
subset_abbrevs_in_order.reverse()
subset_order_map = {subset_abbrev:i for i,subset_abbrev in enumerate(subset_abbrevs_in_order)}



# Producing the line entries that represent connections between the subsets and topics.
line_number = 0
topic_int_list = list(topic_subset_similarity_df.columns)
subset_str_list = list(topic_subset_similarity_df.index)
for subset_abbrev, topic_int in itertools.product(topic_int_list,subset_str_list):
    
    # The weight of the line, extracted from the similarity matrix between subsets and topics built previously.
    weight = topic_subset_similarity_df.loc[topic_int,subset_abbrev]
    
    # The strings that should be used to represent classes, subsets, and topics in a figure or plot.
    subset_str = "{} ({})".format(subset_abbrev, subset_to_desc_map[subset_abbrev].lower())
    tm_lines_dict["subset_str"].extend([subset_str,subset_str])
    tm_lines_dict["class_str"].extend([subset_to_class_map[subset_abbrev],subset_to_class_map[subset_abbrev]])
    topic_str = "Topic {}: ({})".format(topic_renumbered_map[topic_int], "|".join(map_top_words[topic_int]))
    tm_lines_dict["topic_str"].extend([topic_str,topic_str])
    
    # Which line is this, they all have individual numbers so that each line can be its own group in a ggplot object.
    tm_lines_dict["line_number"].extend([line_number,line_number])
    tm_lines_dict["weight"].extend([weight,weight])
    
    # Where should the line start and stop? The horizontal values are arbitrary and just have to match.
    # The vertical values are determined by which subset and topic are being connected to each other.
    tm_lines_dict["x"].extend([0,10])
    tm_lines_dict["y"].extend([subset_order_map[subset_abbrev],topic_order_map[topic_int]])
    
    line_number = line_number+1
    
tm_lines_df = pd.DataFrame(tm_lines_dict)
tm_lines_df.to_csv(os.path.join(OUTPUT_DIR, "topic_subset_lines.csv"), index=False)
tm_lines_df.head(50)

,subset_str,class_str,topic_str,line_number,weight,x,y
0,OBI (other biological interactors),Biological,Topic 42: (resist|kanamycin|similar|morpholog|...,0,0.028782,0,0
1,OBI (other biological interactors),Biological,Topic 42: (resist|kanamycin|similar|morpholog|...,0,0.028782,10,0
2,OBI (other biological interactors),Biological,Topic 41: (lesion|suscept|infect|fungal|chlorot),1,0.017163,0,0
3,OBI (other biological interactors),Biological,Topic 41: (lesion|suscept|infect|fungal|chlorot),1,0.017163,10,1
4,OBI (other biological interactors),Biological,Topic 40: (aba|germin|stomat|insensit|hypersen...,2,0.004004,0,0
5,OBI (other biological interactors),Biological,Topic 40: (aba|germin|stomat|insensit|hypersen...,2,0.004004,10,2
6,OBI (other biological interactors),Biological,Topic 39: (short|hypocotyl|dai|light|height),3,0.003024,0,0
7,OBI (other biological interactors),Biological,Topic 39: (short|hypocotyl|dai|light|height),3,0.003024,10,3
8,OBI (other biological interactors),Biological,Topic 38: (sensit|stress|salt|drought|osmot),4,0.010267,0,0
9,OBI (other biological interactors),Biological,Topic 38: (sensit|stress|salt|drought|osmot),4,0.010267,10,4
